## BC Hydro IPP Supply List Analysis

BC Hydro acquires power from Independent Power Producers (IPPs) to help meet B.C.'s electricity needs.

IPPs develop and operate projects such as wind, water and biomass. IPPs include power production companies, municipalities, First Nations and customers. 

The analysis is carried out on the most recent [IPP supply list](https://www.bchydro.com/content/dam/BCHydro/customer-portal/documents/corporate/independent-power-producers-calls-for-power/independent-power-producers/ipp-supply-list-in-operation.pdf) and [IPP supply map](https://www.bchydro.com/content/dam/BCHydro/customer-portal/documents/corporate/independent-power-producers-calls-for-power/independent-power-producers/ipp-supply-map.pdf) obtained from [BC Hydro Website](https://www.bchydro.com/work-with-us/selling-clean-energy/meeting-energy-needs/how-power-is-acquired.html).

BC Hydro’s integrated system is an interconnected network of transmission lines, distribution lines and
substations linking generating stations to one another and to customers throughout BC Hydro’s service area,
excluding isolated customers who are connected to free-standing generating facilities. Some of BC Hydro’s
customers live in areas that are not served by the integrated system. Local generation serves these
Non-Integrated Areas (NIAs). Unless otherwise indicated, this IRP does not address NIAs. 

Existing supply-side resources include BC Hydro’s Heritage hydroelectric and thermal (natural gas-fired)
generating resources, as well as independent power producer (IPP) facilities delivering electricity to
BC Hydro. Committed supply-side resources are resources for which material regulatory and BC Hydro
executive approvals have been secured

In [1]:
import pandas as pd
import altair as alt

In [2]:
data = pd.read_csv('IPP__REGION_list.csv')

In [3]:
data.columns

Index(['Sub_region', 'Project_name', 'IPP_seller', 'Location', 'Type', 'Year',
       'Call_process', 'Capacity(MW)', 'Energy(GWh/yr)'],
      dtype='object')

#### Initial Description of Data 

In [4]:
len(data)

119

In [5]:
IPP_Capacity = round(data['Capacity(MW)'].sum())
IPP_Capacity

5299

In [6]:
IPP_Energy = round(data['Energy(GWh/yr)'].sum())
IPP_Energy

18662


From the Independent Power Producer (IPP) Supply List - In Operation As of April 1, 2025[[1]](https://www.bchydro.com/content/dam/BCHydro/customer-portal/documents/corporate/independent-power-producers-calls-for-power/independent-power-producers/ipp-supply-list-in-operation.pdf)
- BC Hydro has 119 Electricity Purchase Agreements (EPAs) with IPPs
- In total, these projects are capable of delivering 
    - Approximately 18,662 gigawatt hours of annual supply
    - Approximately 5,298 megawatts of capacity


In [7]:
data.replace({'Sub_region':{'CI':'Central Interior',
                               'LM':'Lower Mainland',
                               'VI':'Vancouver Island',
                               'NC':'North Coast',
                               'EK':'East Kootenay',
                               'PR':'Peace River',
                               'KLN':'Kelly Lake Nicola',
                               'RSAC':'Revelstoke Ashton Creek',
                               'SK':'Selkirk'}}, inplace = True)

In [8]:
peace_list = ['Central Interior','Peace River']

In [9]:
columbia_list = ['Revelstoke Ashton Creek', 'Kelly Lake Nicola', 'Mica', 'Selkirk', 'East Kootenay']

In [10]:
data.loc[data['Sub_region'].isin(columbia_list),'Region'] = 'Columbia Region'
data.loc[data['Sub_region'].isin(peace_list),'Region'] = 'Peace Region'
data.loc[data['Sub_region'] == 'Lower Mainland','Region'] = 'Lower Mainland'
data.loc[data['Sub_region'] == 'North Coast','Region'] = 'North Coast'
data.loc[data['Sub_region'] == 'Vancouver Island','Region'] = 'Vancouver Island'

In [11]:
columns = data.columns.to_list()

In [12]:
for column in columns:
    if data[column].dtype == 'object':
        data[column] = data[column].str.strip()

#### Capacity Availability - Based on type of resource

In [13]:
capacity_IPP = alt.Chart(data).mark_bar(height=20).encode(
                    alt.Y('Type', title='',sort='x'),
                    alt.X('sum(Capacity(MW))', title='Capacity (MW)')).properties(height=300, width=500)
chart_IPP = capacity_IPP + capacity_IPP.mark_text(align='left', dx = 3).encode(text=alt.Text('sum(Capacity(MW))'))
chart_IPP

alt.LayerChart(...)

In [14]:
energy_IPP = alt.Chart(data).mark_bar(height=20,color='green').encode(
                    alt.Y('Type', title='',sort='x'),
                    alt.X('sum(Energy(GWh/yr))', title='Energy (MW)')).properties(height=300, width=500)
chart_IPP = energy_IPP + energy_IPP.mark_text(align='left', dx = 3).encode(text=alt.Text('sum(Energy(GWh/yr))'))
chart_IPP

alt.LayerChart(...)

In [15]:
Capacity_title = alt.TitleParams(
    "Google's stock experiencing heavier fluctuations than competitors",
     subtitle = "Prices have been surging since 2009, but have still not reached an all-time high")


In [16]:
energy_IPP_by_type = alt.Chart(data).mark_bar(height=20).encode(
alt.Y('Type', title='',sort='x'),
alt.X('sum(Energy(GWh/yr))', title='GWh/yr'),
alt.Color('Region', title='', legend=None),
alt.Tooltip('sum(Energy(GWh/yr))')).properties(height=200, width=400).facet('Region', columns = 1).resolve_scale(x='independent',y='independent')
energy_IPP_by_type

alt.FacetChart(...)

In [17]:
capacity_IPP_by_Region = alt.Chart(data).mark_bar(height=20).encode(
alt.Y('Type', title='', sort='x'),
alt.X('sum(Capacity(MW))', title='Capacity (MW)'),
alt.Color('Region', title='',legend=None),
alt.Tooltip('sum(Capacity(MW))')).properties(height=200, width=400).facet('Region', columns = 1).resolve_scale(x='independent',y='independent')
capacity_IPP_by_Region

alt.FacetChart(...)

In [18]:
list_cp = list(data['Call_process'].unique())

In [19]:
# for call_process in list_cp:
#     print(call_process)
#     display(data[data['Call_process']== str(call_process)])
#     print('\n')

In [20]:
data = data.assign(Integration='Integrated')

In [21]:
data.loc[data['Project_name'] == 'Pine Creek', 'Integration'] = 'Non-integrated'
data.loc[data['Project_name'].str.contains('Hluey'), 'Integration'] = 'Non-integrated'
data.loc[data['Project_name'].str.contains('Moresby'), 'Integration'] = 'Non-integrated'
data.loc[data['Project_name'] == 'Kwadacha Bioenergy Project', 'Integration'] = 'Non-integrated'
data.loc[data['Project_name'] == 'Ocean Falls', 'Integration'] = 'Non-integrated'

In [22]:
data_int = data[data['Integration']=='Integrated'].reset_index(drop=True)

In [23]:
data_non_int = data[data['Integration']=='Non-integrated'].reset_index(drop=True)

In [24]:
data_non_int

,Sub_region,Project_name,IPP_seller,Location,Type,Year,Call_process,Capacity(MW),Energy(GWh/yr),Region,Integration
0,North Coast,Pine Creek,XEITL Limited Partnership,Atlin,Storage Hydro,2006,Non-Integrated Areas RFP,2.0,4.6,North Coast,Non-integrated
1,North Coast,Ocean Falls,Boralex Ocean Falls LP,Bella Bella,Non-Storage Hydro,,Non-Integrated Areas RFP,15.0,12.3,North Coast,Non-integrated
2,North Coast,Hluey Lake,MPT Hydro LP,Dease Lake,Storage Hydro,2020,Non-Integrated Areas NEPA,3.0,5.0,North Coast,Non-integrated
3,Peace River,Kwadacha Bioenergy Project,Kwadacha Green Energy Limited Partnership,Fort Ware,Biomass,2016,Non-Integrated Areas RFP,0.5,1.0,Peace Region,Non-integrated
4,North Coast,Moresby Lake,Atlantic Power (Coastal Rivers) Corporation,Sandspit,Storage Hydro,2024,Non-Integrated Areas NEPA,6.0,20.0,North Coast,Non-integrated


In [40]:
round(data_int['Capacity(MW)'].sum())

5272

In [41]:
round(data_non_int['Capacity(MW)'].sum())

26

In [38]:
def function_IPP_Gen_By_Region_By_Type(integration,region,data, ht):
    data_int_type = data[data['Integration']==str(integration)]
    data_region = data_int_type[data_int_type['Region'] == str(region)]
    cap = data_region['Capacity(MW)'].sum()
    gen = data_region['Project_name'].nunique()
    print(region, cap, gen)
    title = 'Capacity of ' + str(integration) + ' IPP Projects '+ str(region) +' by Type'

    capacity_IPP = alt.Chart(data_region, title = title).mark_bar(height = 18).encode(
                    alt.Y('Project_name', sort='x', title = 'IPP Project Name'),
                    alt.X('sum(Capacity(MW))',title = 'Generating Capacity(MW)'),
                    alt.Color('Type', title = 'Type of Generation')).properties(height=int(ht), width = 200)
    energy_CR_Loc_chart = capacity_IPP + capacity_IPP.mark_text(
        color = 'black', align='left', dx = 3).encode(text='sum(Capacity(MW))',color=alt.value('black'))
    return(energy_CR_Loc_chart)

In [26]:
function_IPP_Gen_By_Region_By_Type('Integrated','Lower Mainland',data,750)

Lower Mainland 1202.8999999999999 37


alt.LayerChart(...)

In [27]:
function_IPP_Gen_By_Region_By_Type('Integrated','Columbia Region',data,550)

Columbia Region 1163.49 28


alt.LayerChart(...)

In [28]:
function_IPP_Gen_By_Region_By_Type('Integrated','Vancouver Island',data,450)

Vancouver Island 655.155 20


alt.LayerChart(...)

In [29]:
function_IPP_Gen_By_Region_By_Type('Non-integrated','North Coast',data,120)

North Coast 26.0 4


alt.LayerChart(...)

In [39]:
function_IPP_Gen_By_Region_By_Type('Integrated','North Coast',data,200)

North Coast 1285.0 9


alt.LayerChart(...)

In [31]:
function_IPP_Gen_By_Region_By_Type('Integrated','Peace Region',data,400)

Peace Region 965.8 20


alt.LayerChart(...)

In [32]:
function_IPP_Gen_By_Region_By_Type('Non-integrated','Peace Region',data,100)

Peace Region 0.5 1


alt.LayerChart(...)

In [33]:
title = alt.TitleParams('Capacity of Non-Integrated IPP Generating Stations by Type of Generation')

capacity_IPP = alt.Chart(data_non_int, title = title).mark_bar(height=15).encode(
                    alt.Y('Region', sort='x'),
                    alt.X('sum(Capacity(MW))',title = 'Generating Capacity(MW)'),
                    alt.Color('Type')).properties(height=100,width=500)
energy_CR_Loc_chart = capacity_IPP + capacity_IPP.mark_text(
    color = 'black', align='left', dx = 3).encode(text='sum(Capacity(MW))',color=alt.value('black'))
energy_CR_Loc_chart

alt.LayerChart(...)

In [34]:


capacity_IPP = alt.Chart(data_int).mark_bar(height=15).encode(
                    alt.Y('Region', sort='x'),
                    alt.X('sum(Capacity(MW))',title = 'Generating Capacity(MW)')).properties(height=200,width=300)
energy_CR_Loc_chart = capacity_IPP + capacity_IPP.mark_text(
    color = 'black', align='left', dx = 3).encode(text='sum(Capacity(MW))',color=alt.value('black'))
energy_CR_Loc_chart

alt.LayerChart(...)

In [35]:
capacity_int = data_int.groupby(['Region','Type']).agg({'Capacity(MW)':'sum'}).sort_values(by = ['Region','Capacity(MW)'],ascending = False).reset_index()
for index, row in capacity_int.iterrows():
    print(row['Region'],'&',row['Type'],'&',row['Capacity(MW)'],'\\''\\')

Vancouver Island & Gas Fired Thermal & 275.0 \\
Vancouver Island & Wind & 201.2 \\
Vancouver Island & Non-Storage Hydro & 93.4 \\
Vancouver Island & Biomass & 55.0 \\
Vancouver Island & Storage Hydro & 29.9 \\
Vancouver Island & Biogas & 0.355 \\
Vancouver Island & ERG & 0.3 \\
Peace Region & Wind & 515.8 \\
Peace Region & Biomass & 303.5 \\
Peace Region & Gas Fired Thermal & 120.0 \\
Peace Region & Non-Storage Hydro & 20.5 \\
Peace Region & ERG & 6.0 \\
North Coast & Storage Hydro & 934.2 \\
North Coast & Non-Storage Hydro & 298.8 \\
North Coast & Biomass & 52.0 \\
Lower Mainland & Non-Storage Hydro & 1046.4 \\
Lower Mainland & Biomass & 112.0 \\
Lower Mainland & MSW & 24.8 \\
Lower Mainland & Storage Hydro & 10.9 \\
Lower Mainland & Gas Fired Thermal & 8.8 \\
Columbia Region & Non-Storage Hydro & 509.1 \\
Columbia Region & Storage Hydro & 306.1 \\
Columbia Region & Biomass & 294.9 \\
Columbia Region & Wind & 30.0 \\
Columbia Region & ERG & 16.5 \\
Columbia Region & Biogas & 4.8 \\
Co

In [36]:
capacity_non_int = data_non_int.groupby(['Region','Type']).agg({'Capacity(MW)':'sum'}).sort_values(by = ['Region','Capacity(MW)'],ascending = False).reset_index()
for index, row in capacity_non_int.iterrows():
    print(row['Region'],'&',row['Type'],'&',row['Capacity(MW)'],'\\''\\')

Peace Region & Biomass & 0.5 \\
North Coast & Non-Storage Hydro & 15.0 \\
North Coast & Storage Hydro & 11.0 \\


Energy procurement based on the Call Process

In [37]:
capacity_IPP_by_Region = alt.Chart(data).mark_bar(height=20).encode(
alt.Y('Call_process', title='', sort='x'),
alt.X('sum(Capacity(MW))', title='Capacity (MW)'),
alt.Color('Region', title='',legend=None),
alt.Tooltip('sum(Capacity(MW))')).properties(height=500, width=400).facet('Region', columns = 1).resolve_scale(x='independent',y='independent')
capacity_IPP_by_Region

alt.FacetChart(...)